<h3>ROVENA: Web Scraping</h3>

Está é uma versão reduzida do código, para fins apenas webscraping

In [ ]:
!pip install snscrape

*é necessário dar pip install nas bibliotecas que não são automaticamente importadas pelo Google Colab, como é o caso da snscrape*

In [ ]:
import snscrape.modules.twitter as sntwitter
import datetime as dt
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import smtplib

**Arquivos necessários**

Datasheet com as ofensas em português, separadas por classe: [Download](https://drive.google.com/file/d/1WuxMbhZHPoYBVbVmleyU_Jvv9GS5yb93/view?usp=share_link)<br>
client_secret.json para autenticação com o Google Drive: [Download](https://github.com/ieee-ia-uefs/rovena/blob/main/client_secret.json)

<h2>DataManager</h2> é uma classe é responsável por lidar com o gerenciamento de dados, como o envio de emails e o upload de arquivos para a núvem do Google Sheets. Alguns metodos não utilizados foram removidos, porém a versào do código no [GitHub](https://github.com/ieee-ia-uefs/rovena) está completa.


In [ ]:
class DataManager(object):

    def __init__(self):
        super().__init__()
        self.swear_list = pd.read_csv('swear_list-UPDATED.csv', index_col=False)  # lista de palavras ofensivas
        self.swear_list_category = self.swear_list.columns.tolist()  # categorias das palavras ofensivas
        self.my_email = "pythonmailtest.micael@gmail.com"  # email do remetente
        self.my_password = ""  # senha do email do remetente

    def authenticate_google_sheet(self):
        """
        Essa função é responsável por autenticar o google sheets.
        """
        scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
                 "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
        credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
        client = gspread.authorize(credentials)
        return client

    def upload_to_google_sheet(self, tweets, csv_type=False):
        sheet_name = "Rovena-Results"
        client = self.authenticate_google_sheet()
        spreadsheet = client.open(sheet_name)  # o arquivo que será editado no google sheets
        if csv_type:
            client.import_csv(spreadsheet.id, data=tweets)
        else:
            spreadsheet.values_append(sheet_name, {'valueInputOption': 'RAW'}, {'values': tweets})

    def clear_csv(self):
        df = pd.DataFrame(columns=['Category', 'Text', 'Datetime', 'Tweet_Id',  'Username', 'Url', 'Word'])
        self.upload_to_google_sheet(tweets=df, csv_type=True)

<h2>TweetScraper</h2> é uma classe responsável por lidar com o scraping dos tweets.

In [ ]:
class TweetScraper(object):

    def __init__(self):
        super().__init__()
        self.tweets_list = []  # lista de tweets
        self.tweets_df = pd.DataFrame()  # dataframe com os tweets
        self.MAX_TWEETS = -1

    def scrape_tweets(self, word, category):
        i = 0
        tweets_list = []
        today = dt.datetime.utcnow().date()
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(word).get_items()):
            tweets_list.append([str(category), str(tweet.content), str(tweet.date), str(tweet.id), str(tweet.username), str(tweet.url), str(word)])
            if i > self.MAX_TWEETS:
                break
        return tweets_list

    def clear_list(self):
        self.tweets_list = []

    def put_into_dataframe(self):
        self.tweets_df = pd.DataFrame(self.tweets_list, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Url', 'Palavra'])
        return self.tweets_df

In [ ]:
ts = TweetScraper()
dm = DataManager()

In [ ]:
dm.swear_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Etnia        11 non-null     object
 1   Religião     2 non-null      object
 2   Xenofobia    23 non-null     object
 3   Capacitismo  57 non-null     object
 4   Sexismo      30 non-null     object
 5   Politica     6 non-null      object
 6   Assedio      14 non-null     object
 7   Homofobia    15 non-null     object
 8   Aparencia    6 non-null      object
 9   Outros       35 non-null     object
 10  Calunia      25 non-null     object
 11  Palavrão     355 non-null    object
dtypes: object(12)
memory usage: 33.4+ KB


In [ ]:
dm.swear_list.head(7)

,Etnia,Religião,Xenofobia,Capacitismo,Sexismo,Politica,Assedio,Homofobia,Aparencia,Outros,Calunia,Palavrão
0,criolo,macumbeiro,baianada,aidético,cachorra,comunista,cuzuda,baitola,feia,babaca,ladra,anus
1,crioulo,herege,ciganos,aidética,cadela,corrupta,boazuda,bicha,feio,bacura,ladrao,baba-ovo
2,denegrir,NaN,caipira,aleijado,rampeira,corrupto,bocetuda,bixa,feiosa,barbeiro,ladroeira,babaovo
3,denigrir,NaN,branquelo,aleijada,baleia,facista,bunduda,boiola,feioso,bebum,ladrona,bagos
4,macaca,NaN,japa,analfabeto,vadia,fascista,chupadeira,veada,feioza,besta,ladrão,bisca
5,macaco,NaN,judiar,analfabeta,muie,nazista,puta,veadao,feiozo,bárbaro,corna,bokete
6,mulata,NaN,gringo,anão,merdalher,NaN,rabão,veado,NaN,bêbado,cornagem,bokete


*as partes vazias do datasheet são mostradas pelo Pandas como "NaN", é necessário tratar isso*

In [ ]:
terms = {category: [word for word in dm.swear_list[category] if pd.notnull(word)] for category in dm.swear_list.columns.tolist()}
print(terms)

{'Etnia': ['criolo', 'crioulo', 'denegrir', 'denigrir', 'macaca', 'macaco', 'mulata', 'mulato', 'negro', 'preto', 'jamal'], 'Religião': ['macumbeiro', 'herege'], 'Xenofobia': ['baianada', 'ciganos', 'caipira', 'branquelo', 'japa', 'judiar', 'gringo', 'roceiro', 'tupiniquim', 'turco', 'argentino', 'paraguaio', 'quirguistao', 'boludo', 'zulu', 'baiano', 'carcamano', 'paraiba', 'flango', 'polaca', 'xingling', 'chingchong', '(((eles)))'], 'Capacitismo': ['aidético', 'aidética', 'aleijado', 'aleijada', 'analfabeto', 'analfabeta', 'anão', 'anã', 'apenado', 'apenada', 'ceguinho', 'canceroso', 'debil', 'débil', 'debiloide', 'debilóide', 'deficiente', 'doida', 'doido', 'esclerosado', 'estupida', 'estúpida', 'estupidez', 'estupido', 'estúpido', 'idiota', 'idiotice', 'imbecil', 'inculto', 'leprosa', 'leproso', 'louco', 'maluco', 'maneta', 'mondronga', 'mondrongo', 'mongol', 'mongoloide', 'mongolóide', 'olhota', 'perneta', 'retardada', 'retardado', 'tuberculoso', 'alienado', 'anormal', 'debilmenta

*agora tudo está em um dicionario, as keys são as classes, que possuem como valor uma lista de ofensas, todos os "NaN" foram removidos*

<h2>Web Scraping</h2>

In [ ]:
ts.MAX_TWEETS = 1000

*caso não seja definido um número máximo de tweets para o scraping (no caso acima, 1000 **por categoria**), ele irá procurar até acabar os tweets com aquela palavra, ou seja, quase que infinitamente, nessa situação é necessário definir outra situação de parada, como por exemplo, uma data em especifico, assim iria minerar todos tweets daquele termo, somente em determinada data, o que resultaria em um eventual termino da operação.*

In [ ]:
mined = 0

for category in terms.keys():
    for word in terms[category]:
        list_of_tweets = ts.scrape_tweets(word, category)
        dm.upload_to_google_sheet(list_of_tweets)
        mined += 1
        print(f"Termo {word} da categoria {category} foi minerado com sucesso! ({mined} / 189)")
    print(f"Todos os termos da categoria {category} foram minerados com sucesso!")
print(f"O data sheet foi atualizado com sucesso!")

Termo criolo da categoria Etnia foi minerado com sucesso! (1 / 57)
Termo crioulo da categoria Etnia foi minerado com sucesso! (2 / 57)
Termo denegrir da categoria Etnia foi minerado com sucesso! (3 / 57)
Termo denigrir da categoria Etnia foi minerado com sucesso! (4 / 57)
Termo macaca da categoria Etnia foi minerado com sucesso! (5 / 57)
Termo macaco da categoria Etnia foi minerado com sucesso! (6 / 57)
Termo mulata da categoria Etnia foi minerado com sucesso! (7 / 57)
Termo mulato da categoria Etnia foi minerado com sucesso! (8 / 57)
Termo negro da categoria Etnia foi minerado com sucesso! (9 / 57)
Termo preto da categoria Etnia foi minerado com sucesso! (10 / 57)
Termo jamal da categoria Etnia foi minerado com sucesso! (11 / 57)
Todos os termos da categoria Etnia foram minerado com sucesso!
Termo macumbeiro da categoria Religião foi minerado com sucesso! (12 / 57)
Termo herege da categoria Religião foi minerado com sucesso! (13 / 57)
Todos os termos da categoria Religião foram minera

*no webscraping acima, eu removi as classes "outros" e "ofensa" por estarem muito bagunçadas, deixei somente os termos mais espeficicos*

In [ ]:
import pandas as pd
import numpy as np

# Open a TSV file
df = pd.read_table('tsvfile.tsv')

df.head(5)

,Category,Text,Classification
0,Capacitismo,@gnfbootie29 @tommyinnit @youtube_bad,NaN
1,Etnia,Eu fico impressionado que a molecada da base d...,NaN
2,Sexismo,@Cardoso Sem prostituta e homossexualidade par...,NaN
3,Sexismo,oi eu tenho muie,NaN
4,Homofobia,Vontade de rebola minha bunda acho q só veado dms,NaN


In [ ]:
# Replace all semicolons (;) in the Text column with an empty string ('')
df['Text'] = df['Text'].str.replace(',', '')


In [ ]:
df.head(5)

,Category,Text,Classification
0,Capacitismo,@gnfbootie29 @tommyinnit @youtube_bad,NaN
1,Etnia,Eu fico impressionado que a molecada da base d...,NaN
2,Sexismo,@Cardoso Sem prostituta e homossexualidade par...,NaN
3,Sexismo,oi eu tenho muie,NaN
4,Homofobia,Vontade de rebola minha bunda acho q só veado dms,NaN


In [ ]:
# split the dataframe into groups of 5000 rows
groups = df.groupby(np.arange(len(df))//5000)

# save each group as a separate CSV file
for i, group in groups:
    group.to_csv("{}.csv".format(i))


In [ ]:
df.head(3)

,Category,Text,Offensive,classification
0,Capacitismo,@gnfbootie29 @tommyinnit @youtube_bad,,None
1,Etnia,Eu fico impressionado que a molecada da base d...,,None
2,Sexismo,@Cardoso Sem prostituta e homossexualidade par...,,None


In [ ]:
# create the target folder
!mkdir target_folder

# copy files to the target folder
for i in range(60):
    !cp {i}.csv drive/MyDrive/finalmente_ultimo/{i}.csv


cp: cannot stat '38.csv': No such file or directory
cp: cannot stat '39.csv': No such file or directory
cp: cannot stat '40.csv': No such file or directory
cp: cannot stat '41.csv': No such file or directory
cp: cannot stat '42.csv': No such file or directory
cp: cannot stat '43.csv': No such file or directory
cp: cannot stat '44.csv': No such file or directory
cp: cannot stat '45.csv': No such file or directory
cp: cannot stat '46.csv': No such file or directory
cp: cannot stat '47.csv': No such file or directory
cp: cannot stat '48.csv': No such file or directory
cp: cannot stat '49.csv': No such file or directory
cp: cannot stat '50.csv': No such file or directory
cp: cannot stat '51.csv': No such file or directory
cp: cannot stat '52.csv': No such file or directory
cp: cannot stat '53.csv': No such file or directory
cp: cannot stat '54.csv': No such file or directory
cp: cannot stat '55.csv': No such file or directory
cp: cannot stat '56.csv': No such file or directory
cp: cannot s